<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/RAG/LLM_RAG_sql_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import sqlite3
import torch
import numpy as np #Import numpy
from transformers import AutoTokenizer, AutoModel, pipeline

In [28]:
# Define the model and tokenizer
#model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
#model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf.
401 Client Error. (Request ID: Root=1-67c18b03-2d7a8b177f2ccb104a04a2eb;7ba830a1-7664-4df8-ad4e-91a3e74f30b7)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-chat-hf/resolve/main/config.json.
Access to model meta-llama/Llama-2-7b-chat-hf is restricted. You must have access to it and be authenticated to access it. Please log in.

In [12]:
# Setting up SQLite connection
conn = sqlite3.connect('rag_example.db')
cur = conn.cursor()

In [13]:
# Create table with pgvector extension
cur.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    content TEXT,
    embedding BLOB
);
""")
conn.commit()

In [14]:
# Function to compute embeddings
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output[0].numpy()

In [15]:
# Inserting documents into the database
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "The rain in Spain stays mainly in the plain.",
    "In a hole in the ground there lived a hobbit."
]

In [16]:
for doc in documents:
    embedding = compute_embedding(doc).tobytes()  # Convert numpy array to bytes
    cur.execute("INSERT INTO documents (content, embedding) VALUES (?, ?)", (doc, embedding))
conn.commit()

In [17]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        doc_embedding = torch.tensor(list(embedding), dtype=torch.float32).view(1, -1)
        similarity = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), doc_embedding)
        similarities.append((content, similarity.item()))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [20]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        # Convert the byte embedding back to a numpy array, then a tensor
        doc_embedding = np.frombuffer(embedding, dtype=np.float32)
        doc_embedding = torch.tensor(doc_embedding).view(1, -1) #View is added to ensure correct dimensions
        similarity = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), doc_embedding)
        similarities.append((content, similarity.item()))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [21]:
# Function to generate a response
def generate_response(query):
    retrieved_docs = retrieve_documents(query)
    combined_text = " ".join(retrieved_docs)
    generator = pipeline("text-generation", model="gpt2")
    response = generator(f"Query: {query}\nDocuments: {combined_text}\nAnswer:", max_length=100)
    return response[0]['generated_text']

In [26]:
# Example query
query = "Tell me about foxes and rain."
response = generate_response(query)
print(response)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: Tell me about foxes and rain.
Documents: The quick brown fox jumps over the lazy dog. The quick brown fox jumps over the lazy dog. The rain in Spain stays mainly in the plain.
Answer: The yellow fox does not show emotion. The yellow fox does not show emotion.
Answer: The orange fox is a normal and happy fox. The orange fox is a normal and happy fox.
Answer: The red fox doesn't show emotion and is in a nice


In [11]:

# Close the database connection
cur.close()
conn.close()